## Parametrizing decorators

In [1]:
def repeat(number=3):
    def actual_decorator(func):
        def wrapper(*args, **kwargs):
            result = None
            for _ in range(number):
                result = func(*args, **kwargs)
            return result
        return wrapper
    return actual_decorator

In [5]:
def repeat_class(number=3):
    class wrapper:
        def __init__(self, func):
            self.func = func
        def __call__(self, *args, **kwargs):
            result = None
            for _ in range(number):
                result = self.func(*args, **kwargs)
            return result
    return wrapper

In [12]:
@repeat(number=2)
def boo():
    print("boo")

In [13]:
boo()

boo
boo


In [8]:
@repeat_class(number=5)
def foo():
    print("foo")

In [9]:
foo()

foo
foo
foo
foo
foo


## Introspection preserving decorators

In [17]:
def not_preserving_decorator(func):
    def wrapper(*args, **kwargs):
        """Internal wrapped function documentation"""
        return function(*args, **kwargs)
    return wrapper

In [18]:
@not_preserving_decorator
def function_with_important_docstring():
    """This is important docstring we do not want to lose."""
    return 0 

In [19]:
print(function_with_important_docstring.__name__)
print(function_with_important_docstring.__doc__)

wrapper
Internal wrapped function documentation


In [14]:
from functools import wraps
def preserving_decorator(func):
    @wraps(func)  # save func's __name__ and __doc__ to wrapper
    def wrapper(*args, **kwargs):
        """Internal wrapped function documentation"""
        return function(*args, **kwargs)
    return wrapper

In [15]:
@preserving_decorator
def function_with_important_docstring():
    """This is important docstring we do not want to lose."""
    return 0    

In [16]:
print(function_with_important_docstring.__name__)
print(function_with_important_docstring.__doc__)

function_with_important_docstring
This is important docstring we do not want to lose.


## Usage Examples

### 1. Argument checking

In [27]:
rpc_info = {}
def xmlrpc(in_=(), out_=(type(None), )):
    def _xmlrpc(func):
        # register the signature
        func_name = func.__name__
        rpc_info[func_name] = (in_, out_)
        def _check_types(elements, types):
            """Subfunction that checks the types"""
            if len(elements) != len(types):
                raise TypeError("argument count is wrong")
            for index, (arg, type_) in enumerate(zip(elements, types)):
                if isinstance(arg, type_):
                    continue
                raise TypeError("arg #%d should be %s" % (index, type_))
        # wrapped function
        def __xmlrpc(*args):  # no keywords allowed
            checkable_args = args[1:]  # remove self
            _check_types(checkable_args, in_)
            # running function
            res = func(*args)
            # checking output
            if not type(res) in (tuple, list):
                checkable_res = (res, )
            else:
                checkable_res = res
            _check_types(checkable_res, out_)
            
            # both input and output types are valid
            return res
        return __xmlrpc
    return _xmlrpc

In [28]:
class RPCView:
    @xmlrpc((int, int))  # (int, int) -> None
    def meth1(self, int1, int2):
        print("received %d and %d" % (int1, int2))
    @xmlrpc((str, ), (int, ))
    def meth2(self, phrase):
        print("received %s" % phrase)
        return 12

In [25]:
rpc_info

{'meth1': ((int, int), (NoneType,)), 'meth2': ((str,), (int,))}

In [30]:
my = RPCView()

In [31]:
my.meth1(1, 2)

received 1 and 2


In [32]:
my.meth2(2)

TypeError: arg #0 should be <class 'str'>